In [1]:
"""
TODO:
1. add previous n_bookings as a feature
2. add number rooms booked previously as average
"""

'\nTODO:\n1. add previous n_bookings as a feature\n2. add number rooms booked previously as average\n'

In [2]:
import pandas as pd
import lightgbm
import category_encoders as ce
from sklearn.preprocessing import CategoricalEncoder,OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import datetime

In [3]:
train = pd.read_csv("train_5CLrC8b/train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [4]:
concated = pd.concat([train,test],ignore_index=True)

/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
concated['booking_date'] = pd.to_datetime(concated['booking_date'],dayfirst=True)
concated['checkin_date'] = pd.to_datetime(concated['checkin_date'],dayfirst=True)
concated['checkout_date'] = pd.to_datetime(concated['checkout_date'],dayfirst=True)

In [6]:
concated['booking_month'] = concated['booking_date'].apply(lambda x:x.month)
concated['booking_year'] = concated['booking_date'].apply(lambda x:x.year)
concated['booking_day'] = concated['booking_date'].apply(lambda x:x.day)

concated['checkin_month'] = concated['checkin_date'].apply(lambda x:x.month)
concated['checkin_year'] = concated['checkin_date'].apply(lambda x:x.year)
concated['checkin_day'] = concated['checkin_date'].apply(lambda x:x.day)

concated['checkout_month'] = concated['checkout_date'].apply(lambda x:x.month)
concated['checkout_year'] = concated['checkout_date'].apply(lambda x:x.year)
concated['checkout_day'] = concated['checkout_date'].apply(lambda x:x.day)

In [7]:
concated['booking_dayofweek'] = concated['booking_date'].apply(lambda x:x.dayofweek)

concated['checkin_dayofweek'] = concated['checkin_date'].apply(lambda x:x.dayofweek)

concated['checkout_dayofweek'] = concated['checkout_date'].apply(lambda x:x.dayofweek)

In [8]:
concated['season_holidayed_code_is_na'] = concated.season_holidayed_code.apply(
    lambda x: 1 if pd.isna(x) else 0)
concated['state_code_residence_is_na'] = concated.state_code_residence.apply(
    lambda x: 1 if pd.isna(x) else 0)
concated['season_holidayed_code'] = concated.season_holidayed_code.apply(
    lambda x: 0 if pd.isna(x) else x)
concated['state_code_residence'] = concated.state_code_residence.apply(
    lambda x: 0 if pd.isna(x) else x)

In [9]:
concated['actual_days_booked'] = concated.apply(
    lambda x: (x['checkout_date'] - x['checkin_date']).days, axis=1)

In [10]:
concated['n_rooms_booked'] = concated.apply(
    lambda x: x['roomnights'] / x['actual_days_booked'], axis=1)

In [11]:
concated['person_per_room'] = concated.apply(
    lambda x: (x['numberofadults'] + x['numberofchildren']) / x['n_rooms_booked'] if x['n_rooms_booked'] else 0,
    axis=1)

In [12]:
def count_weekend(row):
    checkin_date = row['checkin_date']
    actual_days_booked = row['actual_days_booked']
    stayed_in_weekend = 0
    for i in range(actual_days_booked):
        new_day = (checkin_date + datetime.timedelta(days=i)).day_name()
        if new_day == "Saturday" or new_day == "Friday" or new_day == "Sunday":
            stayed_in_weekend += 1
    return stayed_in_weekend

In [13]:
concated['n_weekend_days'] = concated.apply(count_weekend,axis=1)

In [14]:
concated['checkin_month']
seasons = []
for i in list(concated['checkin_month']):
    if i >=1 and i<4:
        seasons.append('w')
    if i>=4 and i<8:
        seasons.append('q')
    if i>=8 and i<11:
        seasons.append('e')
    if i>=11 and i<=12:
        seasons.append('t')
concated['ses']= seasons

In [15]:
concated.drop(['reservation_id','booking_date','checkin_date','checkout_date','memberid'],axis=1,inplace=True)

In [16]:

l=['member_age_buckets','cluster_code','reservationstatusid_code','resort_id','ses']

encoder = ce.polynomial.PolynomialEncoder(l)
concated = encoder.fit_transform(concated)

#encoder = OrdinalEncoder(dtype=np.int32,)
#concated[['memberh_age_buckets','cluster_code','reservationstatusid_code','resort_id']] = encoder.fit_transform(
#    concated[['member_age_buckets','cluster_code','reservationstatusid_code','resort_id']]
#)

In [17]:
# concated.to_csv("./concat_train_test_feat_engg.csv",index=False)

In [18]:
concated.drop(['intercept'],axis=1,inplace=True)
concated.head()

,amount_spent_per_room_night_scaled,booking_type_code,channel_code,cluster_code_0,cluster_code_1,cluster_code_2,cluster_code_3,cluster_code_4,main_product_code,member_age_buckets_0,...,checkout_dayofweek,season_holidayed_code_is_na,state_code_residence_is_na,actual_days_booked,n_rooms_booked,person_per_room,n_weekend_days,ses_0,ses_1,ses_2
0,7.706428,1,3,-0.597614,0.545545,-0.372678,0.188982,-0.062994,1,-0.495434,...,4,0,0,1,1.0,2.0,0,-0.670820,0.5,-0.223607
1,6.662563,1,1,-0.597614,0.545545,-0.372678,0.188982,-0.062994,1,-0.495434,...,3,0,0,5,1.0,2.0,2,-0.670820,0.5,-0.223607
2,7.871602,1,1,-0.358569,-0.109109,0.521749,-0.566947,0.314970,1,-0.495434,...,3,0,0,4,1.0,2.0,1,-0.223607,-0.5,0.670820
3,5.344943,1,1,-0.119523,-0.436436,0.298142,0.377964,-0.629941,1,-0.495434,...,1,0,0,5,1.0,4.0,3,-0.670820,0.5,-0.223607
4,7.059346,1,1,-0.119523,-0.436436,0.298142,0.377964,-0.629941,1,-0.495434,...,5,0,0,5,1.0,2.0,1,0.223607,-0.5,-0.670820


In [19]:
X_ =concated

In [20]:
X_.head()

,amount_spent_per_room_night_scaled,booking_type_code,channel_code,cluster_code_0,cluster_code_1,cluster_code_2,cluster_code_3,cluster_code_4,main_product_code,member_age_buckets_0,...,checkout_dayofweek,season_holidayed_code_is_na,state_code_residence_is_na,actual_days_booked,n_rooms_booked,person_per_room,n_weekend_days,ses_0,ses_1,ses_2
0,7.706428,1,3,-0.597614,0.545545,-0.372678,0.188982,-0.062994,1,-0.495434,...,4,0,0,1,1.0,2.0,0,-0.670820,0.5,-0.223607
1,6.662563,1,1,-0.597614,0.545545,-0.372678,0.188982,-0.062994,1,-0.495434,...,3,0,0,5,1.0,2.0,2,-0.670820,0.5,-0.223607
2,7.871602,1,1,-0.358569,-0.109109,0.521749,-0.566947,0.314970,1,-0.495434,...,3,0,0,4,1.0,2.0,1,-0.223607,-0.5,0.670820
3,5.344943,1,1,-0.119523,-0.436436,0.298142,0.377964,-0.629941,1,-0.495434,...,1,0,0,5,1.0,4.0,3,-0.670820,0.5,-0.223607
4,7.059346,1,1,-0.119523,-0.436436,0.298142,0.377964,-0.629941,1,-0.495434,...,5,0,0,5,1.0,2.0,1,0.223607,-0.5,-0.670820


In [21]:
y_ = X_.drop(['amount_spent_per_room_night_scaled'],axis=1)

In [24]:
import numpy as np
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=30, random_state=0).fit(y_)


In [25]:
labels = kmeans.labels_
labels= list(map(lambda x : str(x), labels))
concated['k_means'] = labels

In [26]:
concated.head()

,amount_spent_per_room_night_scaled,booking_type_code,channel_code,cluster_code_0,cluster_code_1,cluster_code_2,cluster_code_3,cluster_code_4,main_product_code,member_age_buckets_0,...,season_holidayed_code_is_na,state_code_residence_is_na,actual_days_booked,n_rooms_booked,person_per_room,n_weekend_days,ses_0,ses_1,ses_2,k_means
0,7.706428,1,3,-0.597614,0.545545,-0.372678,0.188982,-0.062994,1,-0.495434,...,0,0,1,1.0,2.0,0,-0.670820,0.5,-0.223607,6
1,6.662563,1,1,-0.597614,0.545545,-0.372678,0.188982,-0.062994,1,-0.495434,...,0,0,5,1.0,2.0,2,-0.670820,0.5,-0.223607,17
2,7.871602,1,1,-0.358569,-0.109109,0.521749,-0.566947,0.314970,1,-0.495434,...,0,0,4,1.0,2.0,1,-0.223607,-0.5,0.670820,4
3,5.344943,1,1,-0.119523,-0.436436,0.298142,0.377964,-0.629941,1,-0.495434,...,0,0,5,1.0,4.0,3,-0.670820,0.5,-0.223607,2
4,7.059346,1,1,-0.119523,-0.436436,0.298142,0.377964,-0.629941,1,-0.495434,...,0,0,5,1.0,2.0,1,0.223607,-0.5,-0.670820,8


In [27]:
encoder = ce.polynomial.PolynomialEncoder(['k_means'])
concated = encoder.fit_transform(concated)

In [28]:
concated.drop(['intercept'],axis=1,inplace=True)

concated.columns


Index(['amount_spent_per_room_night_scaled', 'booking_type_code',
       'channel_code', 'cluster_code_0', 'cluster_code_1', 'cluster_code_2',
       'cluster_code_3', 'cluster_code_4', 'main_product_code',
       'member_age_buckets_0',
       ...
       'k_means_19', 'k_means_20', 'k_means_21', 'k_means_22', 'k_means_23',
       'k_means_24', 'k_means_25', 'k_means_26', 'k_means_27', 'k_means_28'],
      dtype='object', length=113)

In [29]:
train = concated[:len(train)]
test = concated[-len(test):]

In [30]:
concated.columns

Index(['amount_spent_per_room_night_scaled', 'booking_type_code',
       'channel_code', 'cluster_code_0', 'cluster_code_1', 'cluster_code_2',
       'cluster_code_3', 'cluster_code_4', 'main_product_code',
       'member_age_buckets_0',
       ...
       'k_means_19', 'k_means_20', 'k_means_21', 'k_means_22', 'k_means_23',
       'k_means_24', 'k_means_25', 'k_means_26', 'k_means_27', 'k_means_28'],
      dtype='object', length=113)

In [31]:
all_predictions = []
all_mse = []
X = train.drop(['amount_spent_per_room_night_scaled'], axis=1)
Y = train['amount_spent_per_room_night_scaled']
for i in range(10):
    print(i)
    trainX, testX, trainY, testY = train_test_split(X, Y,test_size=0.15)
    lgb_model = lightgbm.LGBMRegressor(n_estimators=1000)
    lgb_model.fit(
        trainX,
        trainY,
        eval_set=(testX, testY),
        eval_metric='rmse',
        early_stopping_rounds=100,verbose = 0 
    )
    predictions = lgb_model.predict(test.drop(['amount_spent_per_room_night_scaled'], axis=1))
    test_pred = lgb_model.predict(testX)
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    all_mse.append(sqrt(mean_squared_error(testY, test_pred)))
    all_predictions.append(predictions)

0
1
2
3
4
5
6
7
8
9


In [32]:
#internal_pred = np.stack(internal_test_pred,axis=1).mean(axis=1)
final_predictions = np.stack(all_predictions,axis=1).mean(axis=1)

In [33]:
sample_submission['amount_spent_per_room_night_scaled'] = final_predictions
sample_submission.to_csv(
    "poly_enc_1_k_means_fet_seson_included.csv",
    index=False)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(y_test_internal, pred_orig_test))


In [ ]:
np.array(all_mse).mean(axis=0)

In [ ]:
all_mse